<a href="https://colab.research.google.com/github/soujanyarbhat/SWM_MovieRecommenderSystem/blob/main/SAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset
movies = pd.read_csv('/content/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = pd.read_csv('/content/training_set.csv', delimiter = ',')
training_set = np.array(training_set, dtype = 'int64')
test_set = pd.read_csv('/content/test_set.csv', delimiter = ',')
test_set = np.array(test_set, dtype = 'int')

# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

nb_epoch = 256
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
    
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

epoch: 1 loss: tensor(1.3477)
epoch: 2 loss: tensor(1.0101)
epoch: 3 loss: tensor(0.9900)
epoch: 4 loss: tensor(0.9834)
epoch: 5 loss: tensor(0.9802)
epoch: 6 loss: tensor(0.9784)
epoch: 7 loss: tensor(0.9772)
epoch: 8 loss: tensor(0.9766)
epoch: 9 loss: tensor(0.9759)
epoch: 10 loss: tensor(0.9756)
epoch: 11 loss: tensor(0.9752)
epoch: 12 loss: tensor(0.9749)
epoch: 13 loss: tensor(0.9747)
epoch: 14 loss: tensor(0.9746)
epoch: 15 loss: tensor(0.9743)
epoch: 16 loss: tensor(0.9745)
epoch: 17 loss: tensor(0.9742)
epoch: 18 loss: tensor(0.9741)
epoch: 19 loss: tensor(0.9740)
epoch: 20 loss: tensor(0.9739)
epoch: 21 loss: tensor(0.9738)
epoch: 22 loss: tensor(0.9737)
epoch: 23 loss: tensor(0.9735)
epoch: 24 loss: tensor(0.9732)
epoch: 25 loss: tensor(0.9727)
epoch: 26 loss: tensor(0.9721)
epoch: 27 loss: tensor(0.9712)
epoch: 28 loss: tensor(0.9705)
epoch: 29 loss: tensor(0.9701)
epoch: 30 loss: tensor(0.9693)
epoch: 31 loss: tensor(0.9683)
epoch: 32 loss: tensor(0.9677)
epoch: 33 loss: t